<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 5
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

1. Algorithme de Lesk simplifié.
1. Utilisation de word2vec.
1. Classification supervisée (cours 9) utilisant des traits lexicaux les mots en position -1, -2, ..., et +1, +2, ..., par rapport à *interest*.

Les méthodes (1) et (2) n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L’algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La méthode (3) vise à classifier les occurrences de *interest*, les sens étant les classes, en utilisant comme traits les mots du contexte (ce sera de l'apprentissage supervisé).

## 0. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html).  Il se trouve en bas de cette page.  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez brièvement répondre aux questions suivantes :

1. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?
2. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?  Est-ce qu'om considère le pluriel *interests* aussi ?  Que se passe-t-il si une phrase contient plusieurs occurrences du mot ?

In [ ]:
# Veuillez répondre ici (en commentaire) aux questions 1 et 2.

# 1 : 
# * ZIP : https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
# * README : https://www.d.umn.edu/~tpederse/Data/README.int.txt

# 2 :
# Le fichier est composés de phrases qui contiennent le mot "interest" ou "interests", séparées par des '$$'
# Les occurrences de "interest" ou "interests" sont codifiées par interest(s)_1, interest(s)_2, etc. en fonction de leur sens.
# Donc interests est considérée comme un mot différent de interest et est codifiée différemment.
# Exemple des lignes 147 et 149 :
# ...  he  owned  a secret  and  illegal interest_5  in  a beer distributorship  ; plotted hidden  ownership *interests  in  real estate   that  presented  an alleged conflict  of  *interest  ;...
# ...  he  owned  a secret  and  illegal *interest  in  a beer distributorship  ; plotted hidden  ownership interests_5  in  real estate   that  presented  an alleged conflict  of  *interest  ;...
# On peut voir que les phrases avec plusieurs occurences de "interest" ou "interests" sont dédoublées pour chaque occurence.
# Dans chaque phrase on considére qu'une seule occurence en la codifiant, les autres sont remplacées par *interest ou *interests.

3. D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

In [ ]:
# Veuillez répondre ici (en commentaire) à la question.
# Les sens sont définis dans le fichier README.int.txt :
# * Sense 1 =  361 occurrences (15%) - readiness to give attention
# * Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
# * Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
# * Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
# * Sense 5 =  500 occurrences (21%) - a share in a company or business
# * Sense 6 = 1252 occurrences (53%) - money paid for the use of money

4. De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [ ]:
# Veuillez répondre ici (en commentaire) à la question.
# The electronic version of the first edition of Longman's Dictionary of Contemporary English (https://www.ldoceonline.com/)
# Sens 1 = "if you have an interest in something or someone, you want to know or learn more about them"
# Sens 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
# Sens 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
# Sens 4 = "the things that bring advantages to someone or something"
# Sens 5 = "if you have an interest in a particular company or industry, you own shares in it"
# Sens 6 = "the extra money that you must pay back when you borrow money"

5. En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [ ]:
# Il y a 7 synsets 
# * Sense 1 = "S: (n) interest, involvement (a sense of concern with and curiosity about someone or something) "an interest in music""
# * Sense 2 = "S: (n) interest, interestingness (the power of attracting or holding one's attention (because it is unusual or exciting etc.)) "they said nothing of great interest"; "primary colors can add interest to a room""
# * Sense 3 = "S: (n) pastime, interest, pursuit (a diversion that occupies one's time and thoughts (usually pleasantly)) "sailing is her favorite pastime"; "his main pastime is gambling"; "he counts reading among his interests"; "they criticized the boy for his limited pursuits""
# * Sense 4 = "S: (n) sake, interest (a reason for wanting something done) "for your sake"; "died for the sake of his country"; "in the interest of safety"; "in the common interest""
# * Sense 5 = "S: (n) interest, stake ((law) a right or legal share of something; a financial involvement with something) "they have interests all over the world"; "a stake in the company's future""
# * Sense 6 = "S: (n) interest (a fixed charge for borrowing money; usually a percentage of the amount borrowed) "how much interest do you pay on your mortgage?""

6. Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NelsonWork\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from random import randrange

def extract_from_sent_without_stopwords(sent):
    words = nltk.word_tokenize(sent)
    stopwords = nltk.corpus.stopwords.words('english')
    words_without_stopwords = [word for word in words if word.lower() not in stopwords and len(word) > 1]
    return words_without_stopwords

In [39]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
senses1 = [
    "readiness to give attention",
    "quality of causing attention to be given to",
    "activity, etc. that one gives attention to",
    "advantage, advancement or favor",
    "a share in a company or business",
    "money paid for the use of money"
]

senses1 = [*map(extract_from_sent_without_stopwords, senses1)]

print(senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention', 'given'], ['activity', 'etc', 'one', 'gives', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid', 'use', 'money']]


7. En combinant les définitions obtenues aux points (3) et (4) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [16]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses2 = [
    "if you have an interest in something or someone, you want to know or learn more about them " + "readiness to give attention",
    "a quality or feature of something that attracts your attention or makes you want to know more about it" + " quality of causing attention to be given to",
    "an activity that you enjoy doing or a subject that you enjoy studying" + " activity, etc. that one gives attention to",
    "the things that bring advantages to someone or something" + " advantage, advancement or favor",
    "if you have an interest in a particular company or industry, you own shares in it" + " a share in a company or business",
    "the extra money that you must pay back when you borrow money" + " money paid for the use of money"
]

senses2 = [*map(lambda sent: [*set(extract_from_sent_without_stopwords(sent))], senses2)]


print(senses2)


[['someone', 'attention', 'learn', 'interest', 'readiness', 'know', 'something', 'give', 'want'], ['causing', 'attention', 'given', 'know', 'quality', 'something', 'makes', 'want', 'attracts', 'feature'], ['studying', 'etc', 'subject', 'attention', 'one', 'activity', 'enjoy', 'gives'], ['things', 'someone', 'advantage', 'advantages', 'bring', 'something', 'favor', 'advancement'], ['share', 'interest', 'shares', 'industry', 'particular', 'business', 'company'], ['money', 'back', 'extra', 'borrow', 'pay', 'paid', 'use', 'must']]


8. Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Les phrases sont-elles déjà tokenisées ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [12]:
def parse_interests_file(filename):
    file = open(filename, "r")
    content = file.read()
    file.close()
    sentences = content.replace('\n', '').replace("=", '').split("$$")
    return sentences

In [113]:
# Veuillez répondre ici à la question.
sentences = [*map(nltk.word_tokenize, parse_interests_file("interest-original.txt"))]

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

Il y a 2369 phrases.
En voici 3 au hasard :
[['investor', 'interest_1', 'in', 'stock', 'funds', '``', 'has', "n't", 'stalled', 'at', 'all', ',', '``', 'mr.', 'hines', 'maintains', '.'], ["''", 'it', 'is', 'in', 'the', 'western', 'interest_4', 'to', 'see', 'mr.', 'gorbachev', 'succeed', '.'], ['revco', 'insists', 'that', 'the', 'proposal', 'is', 'simply', 'an', '``', 'expression', 'of', 'interest_1', ',', '``', 'because', 'under', 'chapter', '11', 'revco', 'has', '``', 'exclusivity', 'rights', '``', 'until', 'feb.', '28', '.']]


## 1. Algorithme de Lesk simplifié

Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [22]:
# Veuillez répondre ici à la question.  

# Voici quelques fonctions auxiliaires et indications pour vous aider :

# Fonction score : pour deux tableaux de chaînes 'window' et 'sense', 
# retourne combien de chaînes de 'window' sont présentes dans 'sense'.
def score_lesk(window, sense):
    result = 0
    for w in window:
        if w in sense:
            result += 1
    return result

# Fonction max_score_index : retourne la position du score maximal dans le tableau de scores
# (en tirant au sort si plusieurs scores ont la valeur maximale)
def max_score_index(scores):
    m = max(scores)
    p = 0
    pp = []
    for s in scores:
        if s == m:
            pp.append(p)
        p += 1
    if len(pp) == 1:
        return pp[0]
    else:
        return pp[randrange(len(pp))]
            
def wsd_lesk(senses, sentence, window_size = 7):
    # retourne le numéro (de 1 à 6) du sens le plus probable de 'interest' dans la phrase selon la méthode de Lesk
    # window_size = 7 : combien de mots regarder autour de 'interest'

    # à compléter
    for i, word in enumerate(sentence):
        if word.startswith("interest_") or word.startswith("interests_"):
            window = sentence[max(0, i-window_size):min(len(sentence), i+window_size)]
    
    if window is None:
        raise Exception("No interest in sentence")
    return 1+max_score_index([score_lesk(window, sense) for sense in senses]) # les sens sont de 1 à 6

Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [36]:
# Veuillez répondre ici à la question.
def evaluate_wsd(fct_name, senses, sentences, window_size):
    tp = 0
    for sentence in sentences:
        # get the number of the interest in the sentence (interest_1, interest_2, etc.)
        interest_number = [word.split("_")[1] for word in sentence if word.startswith("interest_") or word.startswith("interests_")]
        if len(interest_number) == 0:
            continue
        interest_number = int(interest_number[0])
        predicted_sense = fct_name(senses, sentence, window_size)
        if predicted_sense == interest_number:
            tp += 1
    return tp/len(sentences)

In [45]:
evaluate_wsd(wsd_lesk, senses2, sentences, 7)

0.22836639932460953

En optimisant la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [47]:
def try_parameters(method, sentences, senses_definitions, window_sizes):
    results = []
    for window_size in window_sizes:
        for i in range(len(senses_definitions)):
            results.append({
                "window_size": window_size,
                "senses_definition": i+1,
                "accuracy": evaluate_wsd(method, senses_definitions[i], sentences, window_size)
            })
    
    return results

In [48]:
results = try_parameters(wsd_lesk, sentences, [senses1, senses2], [3, 5, 7, 9, 11, 13, 15, 17, 19, 21])
results.sort(key=lambda result: result["accuracy"], reverse=True)
results

[{'window_size': 15, 'senses_definition': 2, 'accuracy': 0.2524271844660194},
 {'window_size': 19, 'senses_definition': 2, 'accuracy': 0.24060785141409877},
 {'window_size': 21, 'senses_definition': 2, 'accuracy': 0.23934149430139298},
 {'window_size': 17, 'senses_definition': 2, 'accuracy': 0.23596454200084424},
 {'window_size': 9, 'senses_definition': 2, 'accuracy': 0.23469818488813846},
 {'window_size': 7, 'senses_definition': 2, 'accuracy': 0.23300970873786409},
 {'window_size': 13, 'senses_definition': 2, 'accuracy': 0.23258758970029547},
 {'window_size': 11, 'senses_definition': 2, 'accuracy': 0.22625580413676658},
 {'window_size': 5, 'senses_definition': 2, 'accuracy': 0.21950189953566906},
 {'window_size': 19, 'senses_definition': 1, 'accuracy': 0.21359223300970873},
 {'window_size': 3, 'senses_definition': 2, 'accuracy': 0.21063739974672857},
 {'window_size': 15, 'senses_definition': 1, 'accuracy': 0.21021528070915999},
 {'window_size': 17, 'senses_definition': 1, 'accuracy': 

Sense2 donne de meilleurs résultats parce qu'il y a plus de mots clés, mais les résultats sont très mauvais pour les 2 listes... avec un meilleur score à 22%

## 2. Utilisation de word2vec pour la similarité contexte vs. synset

En réutilisant une partie du code de `wsd_lesk`, définissez maintenant une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec**.  On vous encourage à chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.

Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.  La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*. Vous pouvez choisir la taille de ce voisinage (`window_size`).  En cas d'égalité, tirer le sens au sort.

In [1]:
!pip3 install gensim==3.8.3

  Using cached gensim-3.8.3-cp38-cp38-win_amd64.whl (24.2 MB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached Cython-0.29.14-cp38-cp38-win_amd64.whl (1.7 MB)


In [ ]:
import gensim
from gensim.models import KeyedVectors
import gensim.downloader
#w2v_model = gensim.downloader.load("word2vec-google-news-300")

In [3]:
path_to_file = '../labo_4/w2v_vectors.kv'
w2v_vectors = KeyedVectors.load(path_to_file, mmap='r')

In [58]:
import numpy as np

# Veuillez répondre ici à la question.
def wsd_word2vec(senses, sentence, window_size = 7):
    for i, word in enumerate(sentence):
        if word.startswith("interest_") or word.startswith("interests_"):
            window = sentence[max(0, i-window_size):min(len(sentence), i+window_size)]
    
    if window is None:
        raise Exception("No interest in sentence")
    
    cleaned_window = [word for word in window if word in w2v_vectors.vocab]
    if len(cleaned_window) == 0:
        return np.random.randint(0, len(senses)) + 1 
    
    cleaned_senses = [[word for word in sense if word in w2v_vectors.vocab] for sense in senses]

    return 1+max_score_index([w2v_vectors.n_similarity(cleaned_window, sense) for sense in cleaned_senses])

Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec les précédents ?

In [50]:
evaluate_wsd(wsd_word2vec, senses2, sentences, 7)

0.34107218235542425

In [59]:
results = try_parameters(wsd_word2vec, sentences, [senses1, senses2], [3, 5, 7, 9, 11, 13, 15, 17, 19, 21])
results.sort(key=lambda result: result["accuracy"], reverse=True)
results

[{'window_size': 13, 'senses_definition': 2, 'accuracy': 0.48839172646686363},
 {'window_size': 17, 'senses_definition': 2, 'accuracy': 0.4858590122414521},
 {'window_size': 11, 'senses_definition': 2, 'accuracy': 0.4850147741663149},
 {'window_size': 15, 'senses_definition': 2, 'accuracy': 0.4841705360911777},
 {'window_size': 19, 'senses_definition': 2, 'accuracy': 0.4841705360911777},
 {'window_size': 21, 'senses_definition': 2, 'accuracy': 0.48332629801604055},
 {'window_size': 9, 'senses_definition': 2, 'accuracy': 0.4769945124525116},
 {'window_size': 5, 'senses_definition': 2, 'accuracy': 0.47403967918953144},
 {'window_size': 7, 'senses_definition': 2, 'accuracy': 0.47277332207682565},
 {'window_size': 3, 'senses_definition': 2, 'accuracy': 0.4533558463486703},
 {'window_size': 19, 'senses_definition': 1, 'accuracy': 0.35289151540734487},
 {'window_size': 17, 'senses_definition': 1, 'accuracy': 0.35246939636977626},
 {'window_size': 11, 'senses_definition': 1, 'accuracy': 0.352

Les résultats sont bien meilleurs avec word2vec, avec un score de de 48%. On peut aussi voir que la liste 2 donne encore de meilleurs résultats que la liste 1.

## 3. Classification supervisée avec des traits lexicaux
Dans cette partie du labo, vous entraînerez des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le principal défi sera de transformer chaque phrase en un ensemble de traits, pour créer les données en vue des expériences de classification.

Vous utiliserez le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  (Il existe de nombreux autres classifieurs supervisés, par exemple dans la boîte à outils `scikit-learn`.)

De plus, vous devrez séparer les 2368 occurrences en ensembles d'entraînement et de test.

Pour représenter les traits lexicaux (*features*), vous les coderez comme `mot-2`, `mot-1`, `mot+1`, `mot+2`, etc. (fenêtre de taille `2*window_size` autour de *interest*) et vous leur donnerez les valeurs des mots observés aux emplacements respectifs, ou alors `NONE` si la fenêtre dépasse la limite de la phrase.  Vous ajouterez un trait qui est le mot *interest* lui-même, qui peut être au singulier ou au pluriel.  Pour chaque occurrence de *interest*, vous devez donc générer une représentation formelle avec un dictionnaire Python suivi de l'index du sens :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```
L'index du sens servira à l'entraînement, puis elle sera cachée à l'évaluation, et la prédiction du système sera comparée à elle pour dire si elle est correcte ou non.  Vous regrouperez toutes ces entrées dans une liste totale de 2368 éléments appelée `items_with_features_A`.

En partant de la liste des phrases appelée `sentences`(préparée plus haut), veuillez générer ici cette liste, en vous aidant si nécessaire du livre NLTK.

In [116]:
def interest_pos(sentence):
    for i, word in enumerate(sentence):
        if word.startswith("interest_") or word.startswith("interests_"):
            return i
    return -1

def interest_class(interest):
    return interest.split("_")[1]

def featurize_sentence(sentence, window_size):
    pos = interest_pos(sentence)
    if pos == -1:
        return None
    sample_class = interest_class(sentence[pos])
    
    features = {}
    for i in range(-window_size, window_size + 1):
        index = pos + i
        if 0 <= index < len(sentence):
            features["word{}{}".format("-" if i < 0 else "+", abs(i))] = sentence[index] if index != pos else sentence[pos].split("_")[0]
        else:
            features["word{}{}".format("-" if i < 0 else "+", abs(i))] = "None"
    
    return [features, sample_class]


In [126]:
# Veuillez répondre ici à la question.
WINDOW_SIZE = 7

items_with_features_A = [featurize_sentence(sentence, WINDOW_SIZE) for sentence in sentences]
items_with_features_A = [item for item in items_with_features_A if item is not None]

print(len(items_with_features_A))
print(items_with_features_A[151:154])

2368
[[{'word-7': 'None', 'word-6': 'None', 'word-5': 'None', 'word-4': 'None', 'word-3': 'None', 'word-2': 'None', 'word-1': 'investor', 'word+0': 'interest', 'word+1': 'in', 'word+2': 'stock', 'word+3': 'funds', 'word+4': '``', 'word+5': 'has', 'word+6': "n't", 'word+7': 'stalled'}, '1'], [{'word-7': 'None', 'word-6': "''", 'word-5': 'it', 'word-4': 'is', 'word-3': 'in', 'word-2': 'the', 'word-1': 'western', 'word+0': 'interest', 'word+1': 'to', 'word+2': 'see', 'word+3': 'mr.', 'word+4': 'gorbachev', 'word+5': 'succeed', 'word+6': '.', 'word+7': 'None'}, '4'], [{'word-7': 'proposal', 'word-6': 'is', 'word-5': 'simply', 'word-4': 'an', 'word-3': '``', 'word-2': 'expression', 'word-1': 'of', 'word+0': 'interest', 'word+1': ',', 'word+2': '``', 'word+3': 'because', 'word+4': 'under', 'word+5': 'chapter', 'word+6': '11', 'word+7': 'revco'}, '1']]


On souhaite maintenant entraîner un classifieur sur une partie des données, et le tester sur une autre.  Typiquement, on peut garder 80% des données pour l'entraînement et utiliser les 20% restants pour l'évaluation.  Veuillez faire cette division séparément pour chaque sens, pour que les deux ensembles contiennent les mêmes proportions de sens que l'ensemble de départ ("stratification"), et enregistrer les deux sous-ensembles de `items_with_features_A` sous les noms respectifs de `iwf_A_train` et `iwf_A_test`.

In [119]:
from random import shuffle

In [127]:
shuffle(items_with_features_A)

SPLIT = 0.8

iwf_A_train = items_with_features_A[:int(len(items_with_features_A) * SPLIT)]
iwf_A_test  = items_with_features_A[int(len(items_with_features_A) * SPLIT):]
# Veuillez répondre ici à la question.

print(len(iwf_A_train), ' ', len(iwf_A_test))
print(iwf_A_test[:2], iwf_A_test[-2:])

1894   474
[[{'word-7': 'have', 'word-6': 'competed', 'word-5': 'for', 'word-4': 'customers', 'word-3': 'primarily', 'word-2': 'with', 'word-1': 'the', 'word+0': 'interest', 'word+1': 'rates', 'word+2': 'they', 'word+3': 'pay', 'word+4': 'on', 'word+5': 'their', 'word+6': 'deposits', 'word+7': 'and'}, '6'], [{'word-7': 'had', 'word-6': 'fallen', 'word-5': '$', 'word-4': '300', 'word-3': 'million', 'word-2': 'behind', 'word-1': 'in', 'word+0': 'interest', 'word+1': 'payments', 'word+2': '.', 'word+3': 'None', 'word+4': 'None', 'word+5': 'None', 'word+6': 'None', 'word+7': 'None'}, '6']] [[{'word-7': 'None', 'word-6': 'value', 'word-5': 'of', 'word-4': 'the', 'word-3': 'properties', 'word-2': 'and', 'word-1': 'management', 'word+0': 'interests', 'word+1': 'was', 'word+2': "n't", 'word+3': 'disclosed', 'word+4': '.', 'word+5': 'None', 'word+6': 'None', 'word+7': 'None'}, '5'], [{'word-7': 'and', 'word-6': 'to', 'word-5': 'purchase', 'word-4': 'a', 'word-3': '10', 'word-2': '%', 'word-1': 

Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_A_train` et la tester sur `iwf_A_train` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score global que vous obtenez (avec la fonction `accuracy`), et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs (voir la doc NLTK), et pouvez-vous expliquer cet affichage ?

In [133]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.

classifier_A = naivebayes.NaiveBayesClassifier.train(iwf_A_train)
print(nltk.classify.accuracy(classifier_A, iwf_A_test))

0.8860759493670886


Le score est de 88% ce qui est bien meilleur que les précédents.

In [136]:
classifier_A.show_most_informative_features(10)

Most Informative Features
                  word+1 = 'in'                5 : 6      =     74.6 : 1.0
                  word+0 = 'interests'         3 : 1      =     67.6 : 1.0
                  word-1 = 'other'             3 : 6      =     42.4 : 1.0
                  word-7 = 'None'              6 : 3      =     42.1 : 1.0
                  word-6 = 'None'              6 : 3      =     36.6 : 1.0
                  word+2 = '.'                 6 : 1      =     36.4 : 1.0
                  word+1 = 'of'                4 : 6      =     36.0 : 1.0
                  word+4 = 'None'              6 : 2      =     29.8 : 1.0
                  word-5 = 'None'              6 : 3      =     29.6 : 1.0
                  word+2 = ','                 6 : 1      =     26.4 : 1.0


Cet affichage nous montre les traits les plus informatifs, c'est à dire qui permet de départager le sens entre plusieurs. 
Ici on voit que le mot "in" aparait juste après le mot "interest" 74.6 fois plus souvent pour le sens 5 (a share in a company or business) que pour le sens 6 (money paid for the use of money)
Ce qui est logique car on dit souvent "to have interests in X" pour dire qu'on a des parts dans X.

On peut voir aussi que interests au pluriel permet de départager le sens 3 (activity, etc. that one gives attention to) et le sens 1 (readiness to give attention).

In [137]:
# Veuillez recopier ici en conclusion les scores des trois 
# expériences, pour pouvoir les comparer d'un coup d'oeil.

# Lesk => 0.2524271844660194
# Word2Vec => 0.48839172646686363
# Naive Bayes => 0.8860759493670886

## Fin du laboratoire

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.